# Psychotherapy Analysis
This notebook was used to analyze whether certain claims and hypotheses for waiting times in psychotherapy in Germany are true. To do this, we retrieved our own sample by calling about 183 therapists in Germany, with a bigger sample
in Berlin and Brandenburg so that we can also make subgroup analysis for these two federal states

In [2]:
# we first load in some libraries
import pandas as pd
import altair as alt
import numpy as np
import scipy.stats

In [3]:
#we read in the data that we got ourselves
data=pd.read_csv("../RawData/Umfrage_Gewichte.csv")
#and zfill for the federal state code
data["Land"]=data["Land"].astype(str).str.zfill(2)
data

,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Führt gar keine Warteliste,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen
0,01,Schleswig-Holstein,"Norderstedt, Stadt",22846,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,Schleswig-Holstein,"Kiel, Landeshauptstadt",24103,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,Schleswig-Holstein,"Kiel, Landeshauptstadt",24103,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,Schleswig-Holstein,Wentorf bei Hamburg,21465,1,dicht besiedelt,2,Dr. med. Arnd Schweitzer,0.0,NaN,NaN,1.0,NaN,NaN,NaN,ja,25.0,ja,"vor Kurzem erst eröffnet – Erwartung, dass War..."
4,01,Schleswig-Holstein,"Lübeck, Hansestadt",23539,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,16,Thüringen,Gerstungen,99834,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3536,16,Thüringen,Hörselberg-Hainich,99820,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3537,16,Thüringen,Nauendorf,99448,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3538,16,Thüringen,Oberbösa,99718,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are two metrics that matter to us. 
* **The waiting time for a first consultation (Sprechstunde).** Sprechstunde is a first talk between a therapist and a potential patient to figure out whether there is a need for therapy and whether the therapist is the right one to take care of the potential patient. We asked for how long one has to wait until he/she/they get a "Sprechstunde".
* **The waiting time for a therapy (Therapy).** This is the waiting time that passes in between the first consultation and the actual start of a therapy of a patient. 

Theoretically, therapists should offer regularly first consultations within 2 weeks or less in order to enable quick check ups. However, in reality a lot of therapists said that they do not see the benefit of a first consultation, if it means that after unpacking someone's emotional baggage, patients have to wait for months to actually start a therapy. So, some therapists decide to only offer a first consultation, if they have an available time slot for said patient. This leads to the waiting time for a therapy being unexpectedly low while the waiting time for a first consultation seems to be unreasonably high. To have a more fair metric, we also add the two up as an overall waiting time to create a more fair metric.

In [4]:
#then we only take in as a subset all rows where we have an actual entry for the name of a therapist
subset_name=data.dropna(subset=["Praxisname"])
#we turn all data for the waiting time into numerics
subset_name[["Wartezeit auf Sprechstunde","Wartezeit auf Sprechstunde MIN","Wartezeit auf Sprechstunde MAX"]]=subset_name[["Wartezeit auf Sprechstunde","Wartezeit auf Sprechstunde MIN","Wartezeit auf Sprechstunde MAX"]].apply(pd.to_numeric, errors='coerce')
subset_name[["Wartezeit auf Therapie","Wartezeit auf Therapie MIN","Wartezeit auf Therapie MAX"]]=subset_name[["Wartezeit auf Therapie","Wartezeit auf Therapie MIN","Wartezeit auf Therapie MAX"]].apply(pd.to_numeric, errors='coerce')

#and then yield the min for each waiting time for each row
subset_name["Min_Sprechstunde"]=subset_name[["Wartezeit auf Sprechstunde","Wartezeit auf Sprechstunde MIN","Wartezeit auf Sprechstunde MAX"]].min(axis=1)
subset_name["Min_Therapie"]=subset_name[["Wartezeit auf Therapie","Wartezeit auf Therapie MIN","Wartezeit auf Therapie MAX"]].min(axis=1)
#further more we create an overall waiting time "Min_Gesamtwartezeit" by adding the two
subset_name["Min_Gesamtwartezeit"]=subset_name["Min_Sprechstunde"]+subset_name["Min_Therapie"]
subset_name

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,...,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Führt gar keine Warteliste,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
3,01,Schleswig-Holstein,Wentorf bei Hamburg,21465,1,dicht besiedelt,2,Dr. med. Arnd Schweitzer,0.0,NaN,...,NaN,NaN,NaN,ja,25.0,ja,"vor Kurzem erst eröffnet – Erwartung, dass War...",0.0,1.0,1.0
28,01,Schleswig-Holstein,"Lübeck, Hansestadt",23539,1,dicht besiedelt,2,Dipl.-Psych. Marian Schmitt,2.0,NaN,...,NaN,NaN,ja,NaN,30.0,ja,"Führt keine Warteliste, weil die Anfrage so gr...",2.0,4.0,6.0
55,01,Schleswig-Holstein,"Tornesch, Stadt",25436,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Nadine Rohwer,NaN,NaN,...,NaN,NaN,NaN,NaN,26.0,ja,"gesetztlich verpflichtet, Sprechstunde innerha...",4.0,72.0,76.0
58,01,Schleswig-Holstein,Altenholz,24161,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Katrin Brinkmann,1.0,NaN,...,NaN,NaN,NaN,NaN,60.0,ja,Wartezeit Therapieplatz 1 Jahr,1.0,48.0,49.0
82,01,Schleswig-Holstein,Brodersby,24398,1,gering besiedelt,2,Dr. med. Sabine Keppel,24.0,NaN,...,NaN,NaN,NaN,NaN,40.0,ja,Vermittlung über Kassenärztliche Vereinigung k...,24.0,24.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,15,Sachsen-Anhalt,Kabelsketal,6184,15,gering besiedelt,1,Praxis Dipl.-Psych.Eileen-Kristina Fokin • Aho...,3.0,NaN,...,NaN,NaN,NaN,NaN,50.0,Ja,NaN,3.0,24.0,27.0
3472,16,Thüringen,"Jena, Stadt",7743,16,dicht besiedelt,1,Frau Dipl.-Psych. Gudrun Biersack (Einzelpraxi...,1.0,NaN,...,8.0,24.0,NaN,NaN,64.0,Ja,NaN,1.0,8.0,9.0
3515,16,Thüringen,Unterwellenborn,7333,16,mittlere Besiedlungsdichte,2,Herrn Dipl.-Psych. Thomas Ullrich (Einzelpraxi...,12.0,3.0,...,24.0,36.0,NaN,0,80.0,Ja,NaN,3.0,24.0,27.0
3516,16,Thüringen,"Zella-Mehlis, Stadt",98544,16,mittlere Besiedlungsdichte,2,Herrn Dipl.-Psych. Andreas Dräbing (Einzelprax...,12.0,3.0,...,3.0,NaN,NaN,NaN,90.0,Ja,Schneller über Terminvergabe,3.0,3.0,6.0


In [5]:
subset_name[(subset_name["BL"]==11)|(subset_name["BL"]==12)]

,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,...,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Führt gar keine Warteliste,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
1920,11,Berlin,"Berlin, Stadt",Marzahn-Hellersdorf,11,dicht besiedelt,36,Frau Dipl.-Psych. Svetlana Groß Allee der Kosm...,1.0,1.0,...,2.0,4.0,NaN,NaN,80.0,Ja,NaN,1.0,2.0,3.0
1921,11,Berlin,"Berlin, Stadt",Friedrichshain-Kreuzberg,11,dicht besiedelt,36,Frau Dipl.-Psych. Silke Herkner Bänschstraße 6...,1.0,NaN,...,NaN,NaN,Doch,NaN,60.0,Ja,Aktue Fälle kriegen irgendwie immer einen Plat...,1.0,4.0,5.0
1922,11,Berlin,"Berlin, Stadt",Friedrichshain-Kreuzberg,11,dicht besiedelt,36,Frau Dipl.-Psych. Dominique Devreese Hasenheid...,1.0,NaN,...,NaN,NaN,NaN,NaN,35.0,Ja,NaN,1.0,8.0,9.0
1923,11,Berlin,"Berlin, Stadt",Marzahn-Hellersdorf,11,dicht besiedelt,36,Frau Dipl.-Psych. Frauke Bruysten Hönower Stra...,3.0,2.0,...,6.0,24.0,NaN,NaN,30.0,Ja,NaN,2.0,6.0,8.0
1924,11,Berlin,"Berlin, Stadt",Charlottenburg-Wilmersdorf,11,dicht besiedelt,36,Frau Dipl.-Psych. Marianne Grötschel-El Asfar ...,1.0,1.0,...,4.0,6.0,NaN,NaN,45.0,Ja,NaN,1.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128,12,Brandenburg,Burg (Spreewald)/Bórkowy (Błota),3096,12,gering besiedelt,8,Frau Dipl.-Psych. Claudia Trautmann Hauptstr. ...,4.0,NaN,...,NaN,NaN,NaN,NaN,36.0,Nein,NaN,4.0,4.0,8.0
3166,12,Brandenburg,Groß Lindow,15295,12,gering besiedelt,8,Praxis Dipl.-Psych. Gabriele Schaetz Im Techno...,3.0,NaN,...,16.0,28.0,Doch,NaN,60.0,Ja,Erstes Gespräch und dann doch kein Therapieplatz,3.0,16.0,19.0
3169,12,Brandenburg,"Ruhland, Stadt",1945,12,gering besiedelt,8,Praxis Dipl.-Psych. Ilka Hirsch Kirchplatz 1 0...,1.0,1.0,...,NaN,0.0,NaN,6,40.0,Ja,Hat gerade erst die Praxis aufgemacht,1.0,0.0,1.0
3170,12,Brandenburg,Rehfelde,15345,12,gering besiedelt,8,Praxis Patricia Menezes Krüger Georg-Kurtze-St...,4.0,4.0,...,3.0,4.0,NaN,NaN,70.0,Ja,NaN,4.0,3.0,7.0


From our sample, we want to pick the necessary amount of therapists for each federal state and each type of pop density. This is to stratify for size of population in each federal state and to stratify for the amount of people in rural, small town and urban areas. To do this, we go back to a file that we used to estimate the sample number needed for each combination of pop density and federal state. And as our yielded sample is slightly bigger than the needed one, we take a subsample of this dataset for the overall analysis of Germany and for Berlin-Brandenburg

In [6]:
praxenzahl=pd.read_excel("../RawData/AnzahlPraxen.xlsx",sheet_name="Grad_der_Verstädterung",skiprows=6,dtype=str)
praxenzahl_columnlist=["Land","Grad der Verstädterung","Bezeichnung","Fläche in km2",
                       'Bevölkerung insgesamt', 'Bevölkerung männlich', 'Bevölkerung weiblich',"Bevölkerung pro km2",
                       "Fläche_Prozent","Bevölkerung_Prozent","Praxen_gesamt_Sample","Praxen_BB_Sample"]
praxenzahl.columns=praxenzahl_columnlist
praxenzahl_reduced=praxenzahl.dropna(subset=["Grad der Verstädterung","Land"])
praxenzahl_reduced["Praxen_gesamt_Sample"]=pd.to_numeric(praxenzahl_reduced["Praxen_gesamt_Sample"])
praxenzahl_reduced["Praxen_BB_Sample"]=pd.to_numeric(praxenzahl_reduced["Praxen_BB_Sample"])
praxenzahl_reduced["Praxen_gesamt_Sample"]=praxenzahl_reduced["Praxen_gesamt_Sample"].apply(np.ceil)
praxenzahl_reduced["Praxen_BB_Sample"]=praxenzahl_reduced["Praxen_BB_Sample"].apply(np.ceil)
praxenzahl_reduced["Praxen_BB_Sample"]=praxenzahl_reduced["Praxen_BB_Sample"].fillna(0)
praxenzahl_reduced["Sample"]=praxenzahl_reduced["Praxen_BB_Sample"]+praxenzahl_reduced["Praxen_gesamt_Sample"]
praxenzahl_reduced

/usr/local/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Land,Grad der Verstädterung,Bezeichnung,Fläche in km2,Bevölkerung insgesamt,Bevölkerung männlich,Bevölkerung weiblich,Bevölkerung pro km2,Fläche_Prozent,Bevölkerung_Prozent,Praxen_gesamt_Sample,Praxen_BB_Sample,Sample
1,01,01,dicht besiedelt,683.47,835221,405774,429447,1222,4.32457981398745,28.69312491948297,2.0,0.0,2.0
2,01,02,mittlere Besiedlungsdichte,2458.45,1236573,601871,634702,503,15.555566804245172,42.48114398591489,2.0,0.0,2.0
3,01,03,gering besiedelt,12662.39,839081,418004,421077,66,80.11985338176738,28.825731094602137,2.0,0.0,2.0
5,02,01,dicht besiedelt,755.09,1852478,906933,945545,2453,100,100,3.0,0.0,3.0
7,03,01,dicht besiedelt,1708.49,1901290,928927,972363,1113,3.581002070223264,23.75596635488749,3.0,0.0,3.0
8,03,02,mittlere Besiedlungsdichte,16280.35,3801714,1872376,1929338,234,34.12368059160974,47.50111233683696,5.0,0.0,5.0
9,03,03,gering besiedelt,29720.99,2300417,1150153,1150264,77,62.295317338167,28.742921308275548,3.0,0.0,3.0
11,04,01,dicht besiedelt,419.62,680130,336390,343740,1621,100,100,1.0,0.0,1.0
13,05,01,dicht besiedelt,5933.82,9817621,4798658,5018963,1655,17.39488075468048,54.76880790959506,12.0,0.0,12.0
14,05,02,mittlere Besiedlungsdichte,19244.97,6983858,3433386,3550472,363,56.416264443040596,38.960312001236225,9.0,0.0,9.0


In [7]:
praxenzahl

,Land,Grad der Verstädterung,Bezeichnung,Fläche in km2,Bevölkerung insgesamt,Bevölkerung männlich,Bevölkerung weiblich,Bevölkerung pro km2,Fläche_Prozent,Bevölkerung_Prozent,Praxen_gesamt_Sample,Praxen_BB_Sample
0,01,NaN,Schleswig-Holstein,15804.31,2910875,1425649,1485226,184,100,100,3.500539853084776,NaN
1,01,01,dicht besiedelt,683.47,835221,405774,429447,1222,4.32457981398745,28.69312491948297,1.0044142729019006,NaN
2,01,02,mittlere Besiedlungsdichte,2458.45,1236573,601871,634702,503,15.555566804245172,42.48114398591489,1.4870693752732773,NaN
3,01,03,gering besiedelt,12662.39,839081,418004,421077,66,80.11985338176738,28.825731094602137,1.0090562049095984,NaN
4,02,NaN,Hamburg,755.09,1852478,906933,945545,2453,100,100,2.2277401351699333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
68,nur bedingt mit den Vorjahreswerten vegle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,© Daten (im Auftrag der Herausgebergemeinschaf...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,"Statistisches Bundesamt (Destatis), 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#this is how big the samples must be for each subset
samplesizes=praxenzahl_reduced[["Land","Bezeichnung","Praxen_gesamt_Sample","Praxen_BB_Sample"]]
samplesizes.sum()

Land                    0101010203030304050505060606070707080808090909...
Bezeichnung                dicht besiedelt   mittlere Besiedlungsdicht...
Praxen_gesamt_Sample                                                123.0
Praxen_BB_Sample                                                     54.0
dtype: object

In [8]:
# here we create the "allsample" for all of Germany, we use the fixed random state to ensure reproducability
allsample=pd.DataFrame()
for index, row in samplesizes.iterrows():

    subsample = subset_name[(subset_name["Land"]==row["Land"])&(subset_name["Verstädterung"]==row["Bezeichnung"].strip())]
    reduced_subsample=subsample.sample(n=int(row["Praxen_gesamt_Sample"]), random_state=1)
    allsample=pd.concat([allsample,reduced_subsample],axis=0)


In [9]:
# here we create the "berlinbrandenburgsample" for Berlin&Brandenburg

berlinbrandenburgsample=pd.DataFrame()
for index, row in samplesizes.iterrows():

    subsample = subset_name[(subset_name["Land"]==row["Land"])&(subset_name["Verstädterung"]==row["Bezeichnung"].strip())]
    reduced_subsample=subsample.sample(n=int(row["Praxen_BB_Sample"]), random_state=1)
    berlinbrandenburgsample=pd.concat([berlinbrandenburgsample,reduced_subsample],axis=0)


In [10]:
berlinbrandenburgsample.head()

,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,...,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Führt gar keine Warteliste,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
1922,11,Berlin,"Berlin, Stadt",Friedrichshain-Kreuzberg,11,dicht besiedelt,36,Frau Dipl.-Psych. Dominique Devreese Hasenheid...,1.0,NaN,...,NaN,NaN,NaN,NaN,35.0,Ja,NaN,1.0,8.0,9.0
1950,11,Berlin,"Berlin, Stadt",Treptow-Köpenick,11,dicht besiedelt,36,Frau Dipl.-Psych. Margarete Hüsch Lüdersstraße...,1.0,NaN,...,NaN,NaN,NaN,NaN,45.0,Ja,NaN,1.0,4.0,5.0
1923,11,Berlin,"Berlin, Stadt",Marzahn-Hellersdorf,11,dicht besiedelt,36,Frau Dipl.-Psych. Frauke Bruysten Hönower Stra...,3.0,2.0,...,6.0,24.0,NaN,NaN,30.0,Ja,NaN,2.0,6.0,8.0
1941,11,Berlin,"Berlin, Stadt",Neukölln,11,dicht besiedelt,36,Herr Dipl.-Psych. Peter Neubert Gutschmidtstra...,1.0,NaN,...,NaN,NaN,NaN,NaN,30.0,Ja,NaN,1.0,16.0,17.0
1946,11,Berlin,"Berlin, Stadt",Spandau,11,dicht besiedelt,36,Herr Dipl.-Psych. Arnulf Ritter Zimmerstraße 1...,1.0,NaN,...,16.0,20.0,NaN,NaN,30.0,Ja,Kritisiert das Einführen der Sprechstunde,1.0,16.0,17.0


In [11]:
allsample.head()

,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,...,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Führt gar keine Warteliste,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
3,01,Schleswig-Holstein,Wentorf bei Hamburg,21465,1,dicht besiedelt,2,Dr. med. Arnd Schweitzer,0.0,NaN,...,NaN,NaN,NaN,ja,25.0,ja,"vor Kurzem erst eröffnet – Erwartung, dass War...",0.0,1.0,1.0
28,01,Schleswig-Holstein,"Lübeck, Hansestadt",23539,1,dicht besiedelt,2,Dipl.-Psych. Marian Schmitt,2.0,NaN,...,NaN,NaN,ja,NaN,30.0,ja,"Führt keine Warteliste, weil die Anfrage so gr...",2.0,4.0,6.0
55,01,Schleswig-Holstein,"Tornesch, Stadt",25436,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Nadine Rohwer,NaN,NaN,...,NaN,NaN,NaN,NaN,26.0,ja,"gesetztlich verpflichtet, Sprechstunde innerha...",4.0,72.0,76.0
58,01,Schleswig-Holstein,Altenholz,24161,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Katrin Brinkmann,1.0,NaN,...,NaN,NaN,NaN,NaN,60.0,ja,Wartezeit Therapieplatz 1 Jahr,1.0,48.0,49.0
82,01,Schleswig-Holstein,Brodersby,24398,1,gering besiedelt,2,Dr. med. Sabine Keppel,24.0,NaN,...,NaN,NaN,NaN,NaN,40.0,ja,Vermittlung über Kassenärztliche Vereinigung k...,24.0,24.0,48.0


Now we do some general checks and looks at the metrics by using describe and filtering for certain features like rural, urban etc.

In [13]:
allsample.quantile(0.8)

BL                                11.0
Sample                             9.0
Wartezeit auf Sprechstunde        10.0
Wartezeit auf Sprechstunde MIN     8.0
Wartezeit auf Sprechstunde MAX    12.0
Wartezeit auf Therapie            30.4
Wartezeit auf Therapie MIN        24.0
Wartezeit auf Therapie MAX        36.0
Therapieplätze gesamt             81.0
Min_Sprechstunde                   8.0
Min_Therapie                      26.4
Min_Gesamtwartezeit               48.0
Name: 0.8, dtype: float64

In [12]:
allsample.describe()

,BL,Sample,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Therapieplätze gesamt,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
count,123.000000,123.000000,100.00000,61.000000,58.000000,103.000000,58.000000,54.000000,120.000000,123.000000,123.000000,123.000000
mean,7.666667,6.455285,6.50000,6.016393,9.396552,19.106796,15.379310,27.592593,58.358333,6.065041,18.325203,24.390244
std,3.891282,7.004977,9.72189,9.290662,13.768690,19.968069,16.835455,23.286176,38.525948,9.195277,19.937400,21.059881
min,1.000000,1.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000
25%,5.000000,2.000000,1.75000,1.000000,3.000000,4.000000,3.000000,8.750000,30.000000,1.000000,3.500000,8.000000
50%,8.000000,4.000000,3.00000,3.000000,4.500000,12.000000,10.500000,24.000000,49.000000,3.000000,12.000000,18.000000
75%,9.000000,8.000000,8.00000,8.000000,12.000000,24.000000,24.000000,36.000000,78.250000,7.000000,24.000000,34.000000
max,16.000000,36.000000,48.00000,48.000000,96.000000,96.000000,72.000000,96.000000,220.000000,48.000000,96.000000,100.000000


In [14]:
allsample[allsample["Verstädterung"]=="dicht besiedelt"].describe()

,BL,Sample,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Therapieplätze gesamt,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
count,48.000000,48.000000,39.000000,20.000000,19.000000,38.000000,18.000000,19.000000,46.000000,48.000000,48.000000,48.000000
mean,7.354167,8.750000,4.358974,5.000000,8.157895,17.789474,8.500000,23.210526,49.434783,4.583333,15.250000,19.833333
std,3.900298,10.196996,7.912306,6.215431,7.875864,21.378390,8.438358,22.041826,30.683801,7.830202,19.999468,21.351499
min,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,15.000000,0.000000,0.000000,0.000000
25%,5.000000,3.000000,1.000000,1.000000,3.000000,4.000000,2.250000,8.000000,30.000000,1.000000,2.750000,6.750000
50%,6.500000,4.000000,2.000000,2.500000,4.000000,9.000000,7.000000,20.000000,40.000000,2.000000,8.000000,12.500000
75%,10.250000,12.000000,4.000000,6.000000,10.000000,24.000000,12.000000,30.000000,63.000000,4.000000,21.000000,25.000000
max,16.000000,36.000000,48.000000,24.000000,24.000000,96.000000,32.000000,96.000000,150.000000,48.000000,96.000000,100.000000


In [15]:
allsample[allsample["Verstädterung"]=="mittlere Besiedlungsdichte"].describe()

,BL,Sample,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Therapieplätze gesamt,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
count,48.000000,48.000000,38.000000,27.000000,27.000000,41.000000,23.000000,21.000000,48.000000,48.000000,48.000000,48.000000
mean,7.729167,6.000000,9.052632,7.740741,12.740741,20.000000,17.565217,26.095238,69.062500,7.437500,20.145833,27.583333
std,3.818987,3.073566,10.676945,12.154748,18.552612,21.786464,19.530457,24.606716,47.860794,9.836766,21.331217,21.691094
min,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000
25%,5.000000,3.000000,2.000000,1.500000,3.000000,2.000000,3.500000,6.000000,33.500000,2.000000,2.750000,11.500000
50%,8.000000,7.000000,8.000000,4.000000,8.000000,12.000000,12.000000,24.000000,60.000000,4.000000,12.000000,20.000000
75%,9.000000,8.000000,12.000000,8.000000,12.000000,24.000000,24.000000,36.000000,86.250000,8.500000,26.000000,48.000000
max,16.000000,13.000000,48.000000,48.000000,96.000000,72.000000,72.000000,96.000000,220.000000,48.000000,72.000000,82.000000


In [16]:
allsample[allsample["Verstädterung"]=="gering besiedelt"].describe()

,BL,Sample,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Therapieplätze gesamt,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
count,27.000000,27.000000,23.000000,14.000000,12.000000,24.000000,17.000000,14.000000,26.000000,27.000000,27.000000,27.000000
mean,8.111111,3.185185,5.913043,4.142857,3.833333,19.666667,19.705882,35.785714,54.384615,6.259259,20.555556,26.814815
std,4.098155,2.000712,10.281992,5.907715,1.850471,14.245264,18.154768,22.416267,26.095328,10.170064,17.054964,18.526565
min,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,6.000000,19.000000,1.000000,0.000000,2.000000
25%,5.500000,2.000000,2.000000,1.250000,2.000000,5.500000,4.000000,21.000000,31.250000,2.000000,5.000000,12.000000
50%,8.000000,2.000000,3.000000,2.500000,4.000000,24.000000,24.000000,36.000000,50.000000,3.000000,24.000000,26.000000
75%,9.500000,4.500000,4.000000,4.000000,4.250000,25.000000,24.000000,45.000000,67.500000,4.000000,24.000000,36.500000
max,16.000000,8.000000,48.000000,24.000000,8.000000,48.000000,72.000000,96.000000,110.000000,48.000000,72.000000,78.000000


In [17]:
berlinbrandenburgsample[berlinbrandenburgsample["Bezeichnung"]=="Berlin"].describe()

,BL,Sample,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Therapieplätze gesamt,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
count,31.0,31.0,31.000000,7.000000,6.000000,31.000000,11.000000,12.000000,31.000000,31.000000,31.000000,31.000000
mean,11.0,36.0,2.870968,4.285714,4.166667,8.193548,9.818182,20.750000,48.451613,2.774194,7.870968,10.645161
std,0.0,0.0,3.263351,5.313953,1.602082,9.431929,6.539391,12.757351,20.553732,3.283389,9.326457,9.558760
min,11.0,36.0,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,12.000000,1.000000,1.000000,2.000000
25%,11.0,36.0,1.000000,1.500000,3.250000,3.000000,6.000000,11.000000,32.500000,1.000000,3.000000,5.000000
50%,11.0,36.0,2.000000,2.000000,4.000000,4.000000,8.000000,22.000000,45.000000,1.000000,4.000000,8.000000
75%,11.0,36.0,3.000000,4.000000,5.500000,12.000000,14.000000,24.000000,60.000000,2.500000,10.000000,13.500000
max,11.0,36.0,12.000000,16.000000,6.000000,48.000000,24.000000,48.000000,100.000000,12.000000,48.000000,50.000000


In [18]:
berlinbrandenburgsample[berlinbrandenburgsample["Bezeichnung"]=="Brandenburg"].describe()

,BL,Sample,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Therapieplätze gesamt,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
count,23.0,23.000000,23.000000,7.000000,7.000000,22.000000,9.000000,10.000000,23.000000,23.000000,23.000000,23.000000
mean,12.0,9.956522,3.521739,3.285714,5.285714,15.954545,15.222222,30.900000,51.000000,3.347826,15.782609,19.130435
std,0.0,3.067102,3.028194,1.496026,2.058663,16.087491,14.245857,28.380157,24.736796,2.870044,15.739229,15.401607
min,12.0,6.000000,0.000000,2.000000,3.000000,0.000000,2.000000,3.000000,15.000000,0.000000,0.000000,2.000000
25%,12.0,8.000000,2.000000,2.000000,4.000000,3.250000,4.000000,8.500000,32.500000,2.000000,3.500000,7.500000
50%,12.0,8.000000,3.000000,3.000000,4.000000,12.000000,12.000000,26.000000,45.000000,3.000000,12.000000,15.000000
75%,12.0,13.000000,4.000000,4.000000,7.000000,24.000000,16.000000,45.000000,70.000000,4.000000,24.000000,26.500000
max,12.0,13.000000,15.000000,6.000000,8.000000,48.000000,48.000000,96.000000,103.000000,15.000000,48.000000,52.000000


And we look at the distribution between groups and for different measurements

In [19]:
alt.Chart(allsample).mark_bar().encode(
    alt.X("Min_Sprechstunde:Q"),
    y='count()',
tooltip=['Min_Sprechstunde:Q']
)

alt.Chart(...)

In [20]:
alt.Chart(allsample).mark_bar().encode(
    alt.X("Min_Therapie:Q"),
    y='count()',
tooltip=['Min_Therapie:Q']
)

alt.Chart(...)

In [21]:
alt.Chart(allsample).mark_bar().encode(
    alt.X("Min_Gesamtwartezeit:Q"),
    y='count()',
tooltip=['Min_Gesamtwartezeit:Q']
)

alt.Chart(...)

In [22]:
alt.Chart(allsample).mark_bar().encode(
    alt.X("Min_Sprechstunde:Q"),
    alt.Y('count()'),
    

    column= alt.Column(
    'Verstädterung:N',
    sort=["dicht besiedelt", "mittlere Besiedlungsdichte", "gering besiedelt"])
)

alt.Chart(...)

In [23]:
alt.Chart(allsample).mark_bar().encode(
    alt.X("Min_Therapie:Q"),
    y='count()',
    column= alt.Column(
    'Verstädterung:N',
    sort=["dicht besiedelt", "mittlere Besiedlungsdichte", "gering besiedelt"])
)

alt.Chart(...)

In [24]:
alt.Chart(allsample).mark_bar().encode(
    alt.X("Min_Gesamtwartezeit:Q"),
    y='count()',
    column= alt.Column(
    'Verstädterung:N',
    sort=["dicht besiedelt", "mittlere Besiedlungsdichte", "gering besiedelt"])
)

alt.Chart(...)

In [25]:
alt.Chart(berlinbrandenburgsample).mark_bar().encode(
    alt.X("Min_Therapie:Q"),
    y='count()',
    column='Bezeichnung:N'
)

alt.Chart(...)

In [26]:
alt.Chart(berlinbrandenburgsample).mark_bar().encode(
    alt.X("Min_Sprechstunde:Q"),
    y='count()',
    column='Bezeichnung:N'
)

alt.Chart(...)

In [27]:
alt.Chart(berlinbrandenburgsample).mark_bar().encode(
    alt.X("Min_Gesamtwartezeit:Q"),
    y='count()',
    column='Bezeichnung:N'
)

alt.Chart(...)

As we see that the distributions are rather non-normal, we will use non-parametric tests as much as possible. Also, we will now divide the two datasets into two comparisons. rural vs. urban and Berlin vs. Brandenburg

In [28]:
berlin=berlinbrandenburgsample[berlinbrandenburgsample["Bezeichnung"]=="Berlin"]
brandenburg=berlinbrandenburgsample[berlinbrandenburgsample["Bezeichnung"]=="Brandenburg"]
stadt=allsample[allsample["Verstädterung"]=="dicht besiedelt"]
land=allsample[allsample["Verstädterung"]=="gering besiedelt"]

There are three topics that we want to cover:
* 1. A health insurance company claimed in 2020 that less than 16 percent of their respondents said they had to wait 8 or more weeks after a first consultation to start a therapy. On the other hand the association of therapists claim that 40 percent of patients that needed to be treated had to wait at least three to nine months to start a therapy after having had a first consultation. We want to see which or if any of these two claims are true.
* 2. A lot of studies showed a substantial difference between therapists per capita in rural and urban regions. We want to see whether this has an effect in general on waiting times that we can see.
* 3. It has been reported that Berlin had by some metrics too many therapists while Brandenburg is one of the regions with least therapists per capita. We also want to see whether this holds up.


## Claim 1

In [45]:
from scipy import stats
numberoflongwaiter_Krankenkasse=len(allsample[allsample["Min_Therapie"]>=8])
# 3-9 Monate
numberoflongwaiter_BPTK=len(allsample[(allsample["Min_Therapie"]>=12) & (allsample["Min_Therapie"]<=36)])

samplesize=len(allsample)
claimKrankenkasse=0.16
claimBPTK=0.4
stats.binom_test(numberoflongwaiter_Krankenkasse, n=samplesize, p=claimKrankenkasse, alternative='greater')

2.535264821743819e-30

In [46]:
stats.binom_test(numberoflongwaiter_BPTK, n=samplesize, p=claimBPTK, alternative='greater')

0.7509336086982291

So, we can reject the first hypothesis by using a binomial test. Waiting times seem to be much longer than claimed by the health insurers. On the other hand, we cannot reject the numbers from the association of therapists.

## Claim 2

In [31]:
from scipy.stats import mannwhitneyu
mannwhitneyu(stadt["Min_Gesamtwartezeit"], land["Min_Gesamtwartezeit"],alternative = 'less')

MannwhitneyuResult(statistic=460.5, pvalue=0.019457180351305437)

This data supports the hypothesis that the difference between urban and rural in waiting times is signficant in all of Germany

## Claim 3

In [34]:
from scipy.stats import mannwhitneyu
mannwhitneyu(berlin["Min_Gesamtwartezeit"], brandenburg["Min_Gesamtwartezeit"],alternative = 'less')

MannwhitneyuResult(statistic=217.5, pvalue=0.007597639559878934)

This data supports the hypothesis that waiting times are significantly lower in Berlin than in Brandenburg

# Outro
We also check whether the claims regarding Berlin and Brandenburg are true when we do not subsample.

In [35]:
berlin_unsampled=subset_name[(subset_name["BL"]==11)]
brandenburg_unsampled=subset_name[(subset_name["BL"]==12)]

In [47]:
from scipy.stats import mannwhitneyu
mannwhitneyu(berlin_unsampled["Min_Gesamtwartezeit"], brandenburg_unsampled["Min_Gesamtwartezeit"],alternative = 'less')

MannwhitneyuResult(statistic=403.5, pvalue=0.03524959959341699)

And we also check the claims of the health insurance and the psychotherapists with wilcoxon tests

In [48]:
from scipy.stats import wilcoxon
HypKrankenkasse=8

In [49]:
wilcoxon(allsample["Min_Therapie"]-HypKrankenkasse, zero_method = 'wilcox', correction = False)

WilcoxonResult(statistic=1649.0, pvalue=2.3832672013975363e-06)

In [50]:
HypBPTK=6*4

In [51]:
wilcoxon(allsample["Min_Therapie"]-HypBPTK, zero_method = 'wilcox', correction = False)

WilcoxonResult(statistic=1836.5, pvalue=0.005501501476221251)

These additional tests confirm the earlier findings